In [ ]:
!pip install -U langchain langchain-community faiss-cpu gradio pypdf pandas beautifulsoup4 huggingface-hub

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader, TextLoader, WebBaseLoader

def load_pdf(path): return PyPDFLoader(path).load()
def load_csv(path): return CSVLoader(file_path=path).load()
def load_text(path): return TextLoader(path).load()
def load_url(url): return WebBaseLoader(url).load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
docs = load_text("sample.txt")  # Replace with your actual file
chunks = split_documents(docs)

vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
import gradio as gr
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature": 0.2, "max_length": 256})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

def ask_question(query):
    return qa_chain.run(query)

iface = gr.Interface(fn=ask_question, inputs="text", outputs="text", title="LangChain RAG Chatbot")
iface.launch(share=True)